In [8]:
from database_handler import DatabaseHandler
from preprocessing import clean_year_column, impute_km_with_random_forest
from dataframe_transformer import DataFrameTransformer

Reading the data from PostgreSQL

In [9]:
# Create an instance of the DatabaseHandler
db_handler = DatabaseHandler(
    host="localhost",
    port="5432",
    dbname="Data_challenge_REVO",
    user="postgres",
    password="perrito"
)

In [10]:
# Connect to the database
db_handler.connect()

# Load data from tables to dataframes
df_trips = db_handler.load_data('trips')
df_region = db_handler.load_data('region')
df_population = db_handler.load_data('population')
df_urbanization_level = db_handler.load_data('urbanization_level')
df_travel_mode = db_handler.load_data('travel_mode')
df_travel_motives = db_handler.load_data('travel_motives')

# Close the database connection
db_handler.close()

Cleaning

In [11]:
# Cleaning format of the year column
df_trips = clean_year_column(df_trips, 'periods')

Question 1: Show the total number of trips for people who went grocery shopping grouped by travel method and level of urbanization
across all the years.

In [12]:
# Create an instance of DataFrameTransformer with the original dataframe
transformer1 = DataFrameTransformer(df_trips)

# Apply transformations
df_grocery_shopping_grouped_trips = (
    transformer1
    .filter_by_column_value('travelmotives', 2030200) # Filter for grocery shopping
    .merge_multiple_dataframes([
        (df_travel_mode, 'travelmodes', 'code', 'left'), # Merge with travel mode data
        (df_region, 'regioncharacteristics', 'code', 'left'), # Merge with region data
        (df_urbanization_level, 'region', 'provinces', 'left') # Merge with urbanization level data
    ])
    # Address missing values after the merge, since a match was not found for some codes
    .fill_missing_categorical_values(['mode', 'level_urbanization']) 
    # Group by travel method and level of urbanization across all the years
    # The total number of trips was calculated using a sum() of the previously groupped categories
    .group_and_sum(['mode', 'level_urbanization', 'periods'], 'trip_in_a_year', 'total_number_of_trips')
    # Rearrange categories to improve readability of the dataframe
    .reorder_categories('mode', ['Other', 'Unknown'])
    # Create a column with the desired ordered of the labels, to help during the visualization tasks in PowerBI
    .apply_mappings()
    .get_dataframe()
)

# Save results
df_grocery_shopping_grouped_trips.to_csv('df_grocery_shopping_grouped_trips.csv', index=False)

Question 2: Show the users in west Netherlands who travelled the most by bike (in terms of kms) to go to work.

In [13]:
# Create an instance of DataFrameTransformer with the original dataframe
transformer2 = DataFrameTransformer(df_trips)

# Apply transformations
df_top_bikers_west_Netherlands = (
    transformer2
    .filter_by_column_value('travelmodes', 'A018984')  # Filter for bike trips
    .filter_by_column_value('travelmotives', 2030170)  # Filter for trips to work
    .merge_multiple_dataframes([
        (df_region, 'regioncharacteristics', 'code', 'left'), # Merge with region data
        (df_urbanization_level, 'region', 'provinces', 'left') # Merge level of urbanization data
    ])
    .filter_by_column_value('area', 'West-Nederland (LD)')  # Filter for region of interest
    # The total km travelled was calculated using a sum() of the previously groupped categories
    .group_and_sum(['userid'], 'km_travelled_in_a_year', 'total_km_travelled_in_a_year')
    .sort('total_km_travelled_in_a_year', ascending=False)
    .get_dataframe()
    .head(50)
)

# Save results 
df_top_bikers_west_Netherlands.to_csv('df_top_bikers_west_Netherlands.csv', index=False)

Data imputation

Note: for question 3, after applying the conditions requested, the number of motives of travel was very reduced. So, I explored the use of the columns 'hours_travelled_in_a_year' and 'travelmodes' to predict the corresponding 'Km travelled in a year' and get some more data. 

Nevertheless, the number of rows was still very small an this approach had no impact. 

In [ ]:
# Imputes missing 'Km travelled in a year' values in a DataFrame using RandomForestRegressor.
# Only for cases where 'hours_travelled_in_a_year' and 'travelmodes' are present. 
df_trips_imputed = impute_km_with_random_forest(df_trips)

Question 3: Taking the top 8 users above 35 years old who travel the most km by bike, show the 3 least common motives of travel in year 2022.

In [15]:
# Create an instance of DataFrameTransformer with the original dataframe
transformer3 = DataFrameTransformer(df_trips_imputed)

# Apply transformations to identify top 8 bikers
df_top_bikers = (
    transformer3
    .filter_by_column_value('population', 'A048709') # Filter for population over 35 years old
    .filter_by_column_value('travelmodes', 'A018984') # Filter for bike trips
    .group_and_sum(['userid'], 'km_travelled_in_a_year', 'total_km_travelled')
    .sort('total_km_travelled', ascending=False)
    .get_dataframe()
    .nlargest(8, 'total_km_travelled')['userid']
)

# Create another instance for processing trips of top bikers in 2022
transformer4 = DataFrameTransformer(df_trips_imputed)

# Apply transformations to process trips of top bikers in 2022 and count travel motives
df_least_common_motives = (
    transformer4
    .filter_by_column_value('periods', '2022') # Filter for trips in 2022
    .filter_by_values_in_list('userid', df_top_bikers) # Filter the rows corresponding to the 8 top bikers
    .merge_multiple_dataframes([
        (df_travel_motives, 'travelmotives', 'code', 'left') # Merge with travel motive data
    ])
    .count_values('motive', 'motive_count')
    .get_dataframe()
)

# Save results
df_least_common_motives.to_csv('df_least_common_motives.csv', index=False)